In [19]:
import random
import time
from tqdm import tqdm

import torch
from datasets import load_dataset
from transformers import AutoTokenizer, AutoModelForSeq2SeqLM

In [20]:
MODEL_NAME = "google/flan-t5-small"
LABELS = ["World", "Sports", "Business", "Sci/Tech"]
K_FEW = 3
K_ONE = 1
LIMIT_TEST_SAMPLES = 500
MAX_OUTPUT_TOKENS = 50
TEMPERATURE = 0.0
NUM_BEAMS = 1
DELAY_BETWEEN_REQUESTS = 0.1

In [21]:
# Подготовка данных
data = load_dataset("ag_news")
train_data, test_data = data["train"], data["test"]
print(f"Train size: {len(train_data)}  |  Test size: {len(test_data)}")

Train size: 120000  |  Test size: 7600


In [22]:
# Отбор примеров
shuffled_indices = list(range(len(train_data)))
random.shuffle(shuffled_indices)
few_examples = [train_data[i] for i in shuffled_indices[:K_FEW]]
one_example = few_examples[0]

print("\nFew-shot примеры:")
for e in few_examples:
    label = LABELS[e["label"]]
    print(f"  → [{label}] {e['text'][:70].strip()}...")

print(f"\nOne-shot пример:\n  → [{LABELS[one_example['label']]}] {one_example['text'][:70].strip()}...\n")



Few-shot примеры:
  → [Sports] First-night nerves for England, France England and France opened their...
  → [World] Hundreds Held in Russian School; 8 Killed BELSAN, Russia - More than a...
  → [Sports] Broadhurst Honors and Obeys by Taking Madrid Open Lead  MADRID (Reuter...

One-shot пример:
  → [Sports] First-night nerves for England, France England and France opened their...



In [23]:
# Загрузка модели
print(f"Загрузка модели {MODEL_NAME}...")
tokenizer = AutoTokenizer.from_pretrained(MODEL_NAME)
model = AutoModelForSeq2SeqLM.from_pretrained(MODEL_NAME).to("cpu").eval()
print("Модель загружена (CPU).\n")


Загрузка модели google/flan-t5-small...
Модель загружена (CPU).



In [24]:
# Построение подсказок
def generate_prompt_zero_shot(article: str) -> str:
    return (
        f"Classify the article into one of the following: {', '.join(LABELS)}.\n"
        f"Text:\n\"\"\"\n{article.strip()}\n\"\"\"\nLabel:"
    )

def generate_prompt_one_shot(article: str, example: dict) -> str:
    return (
        f"Classify the article into one of the following: {', '.join(LABELS)}.\n"
        f"Example:\n\"\"\"\n{example['text'].strip()}\n\"\"\"\nLabel: {LABELS[example['label']]}\n---\n"
        f"Now classify this:\n\"\"\"\n{article.strip()}\n\"\"\"\nLabel:"
    )

def generate_prompt_few_shot(article: str, examples: list) -> str:
    prompt = f"Classify the article into one of the following: {', '.join(LABELS)}.\n"
    for ex in examples:
        prompt += (
            f"Example:\n\"\"\"\n{ex['text'].strip()}\n\"\"\"\nLabel: {LABELS[ex['label']]}\n---\n"
        )
    prompt += f"Now classify this:\n\"\"\"\n{article.strip()}\n\"\"\"\nLabel:"
    return prompt

In [25]:
# Классификация текста
def get_prediction(prompt: str) -> str:
    encoded = tokenizer(prompt, return_tensors="pt", truncation=True, max_length=512).to("cpu")
    with torch.no_grad():
        output = model.generate(
            **encoded,
            max_length=MAX_OUTPUT_TOKENS,
            temperature=TEMPERATURE,
            num_beams=NUM_BEAMS,
            early_stopping=True
        )
    decoded = tokenizer.decode(output[0], skip_special_tokens=True).strip()
    for label in LABELS:
        if label.lower() in decoded.lower():
            return label
    return decoded


In [26]:
# Запуск режима
def evaluate_mode(mode: str):
    assert mode in {"zero", "one", "few"}, "Mode should be 'zero', 'one', or 'few'"
    predictions, targets = [], []

    N = min(LIMIT_TEST_SAMPLES, len(test_data))
    print(f"\nРежим: {mode.upper()}-SHOT — {N} тестовых примеров")

    for i in tqdm(range(N), desc=f"{mode}-shot"):
        article = test_data[i]["text"]
        label = LABELS[test_data[i]["label"]]
        targets.append(label)

        if mode == "zero":
            prompt = generate_prompt_zero_shot(article)
        elif mode == "one":
            prompt = generate_prompt_one_shot(article, one_example)
        else:
            prompt = generate_prompt_few_shot(article, few_examples)

        pred = get_prediction(prompt)
        predictions.append(pred)
        time.sleep(DELAY_BETWEEN_REQUESTS)

    return targets, predictions

def accuracy(y_ref, y_pred):
    return sum(yt.lower() == yp.lower() for yt, yp in zip(y_ref, y_pred)) / len(y_ref)

In [27]:
# Результаты
results = {}

for mode in ["zero", "one", "few"]:
    y_true, y_pred = evaluate_mode(mode)
    acc = accuracy(y_true, y_pred)
    results[mode] = (y_true, y_pred, acc)
    print(f"Accuracy ({mode}-shot): {acc:.3f}")

# Демонстрация предсказаний
print("\nНесколько примеров (few-shot):")
for i in range(min(5, len(results["few"][0]))):
    true_label = results["few"][0][i]
    predicted_label = results["few"][1][i]
    article = test_data[i]['text'][:60].strip()
    print(f"{i+1}) {true_label} → {predicted_label} | {article}...")



Режим: ZERO-SHOT — 500 тестовых примеров


zero-shot:   7%|██                             | 34/500 [00:07<01:30,  5.13it/s]The following generation flags are not valid and may be ignored: ['temperature', 'early_stopping']. Set `TRANSFORMERS_VERBOSITY=info` for more details.


zero-shot:  14%|████▎                          | 69/500 [00:14<01:24,  5.11it/s]The following generation flags are not valid and may be ignored: ['temperature', 'early_stopping']. Set `TRANSFORMERS_VERBOSITY=info` for more details.


zero-shot:  21%|██████▏                       | 104/500 [00:20<01:14,  5.32it/s]The following generation flags are not valid and may be ignored: ['temperature', 'early_stopping']. Set `TRANSFORMERS_VERBOSITY=info` for more details.


zero-shot:  28%|████████▎                     | 139/500 [00:27<01:07,  5.38it/s]The following generation flags are not valid and may be ignored: ['temperature', 'early_stopping']. Set `TRANSFORMERS_VERBOSITY=info` for more details.


...

few-shot:  84%|█████████████████████████▉     | 419/500 [03:17<00:34,  2.35it/s]The following generation flags are not valid and may be ignored: ['temperature', 'early_stopping']. Set `TRANSFORMERS_VERBOSITY=info` for more details.


few-shot:  91%|████████████████████████████▏  | 454/500 [03:34<00:19,  2.33it/s]The following generation flags are not valid and may be ignored: ['temperature', 'early_stopping']. Set `TRANSFORMERS_VERBOSITY=info` for more details.


few-shot:  98%|██████████████████████████████▎| 489/500 [03:58<00:03,  3.23it/s]The following generation flags are not valid and may be ignored: ['temperature', 'early_stopping']. Set `TRANSFORMERS_VERBOSITY=info` for more details.


few-shot: 100%|███████████████████████████████| 500/500 [04:01<00:00,  2.07it/s]

Accuracy (few-shot): 0.704

Несколько примеров (few-shot):
1) Business → Business | Fears for T N pension after talks Unions representing worker...
2) Sci/Tech → Science/Tech --- | The Race is On: Second Private Team Sets Launch Date for Hum...
3) Sci/Tech → Science --- | Ky. Company Wins Grant to Study Peptides (AP) AP - A company...
4) Sci/Tech → World | Prediction Unit Helps Forecast Wildfires (AP) AP - It's bare...
5) Sci/Tech → Science/Tech --- | Calif. Aims to Limit Farm-Related Smog (AP) AP - Southern Ca...


## Вывод

Zero-shot оказался наиболее точным
На удивление, наилучшая точность была достигнута в zero-shot режиме. Вероятно, это связано с тем, что подсказка в таком случае получается максимально короткой и сконцентрированной. В отличие от few-shot, где добавление нескольких примеров может приводить к усечению текста при токенизации (из-за ограничения длины входа). Это особенно критично для модели flan-t5-small, у которой ограниченные возможности обработки длинного контекста. Более того, большое количество примеров может сбивать модель — она хуже различает, где заканчиваются обучающие образцы и начинается новая статья.

Некоторые метки интерпретируются иначе, но по смыслу верно
Модель нередко предсказывает "Science" вместо ожидаемой метки "Sci/Tech". Хотя это не является точным совпадением с одной из четырёх категорий AG News, по сути, это корректный ответ. Однако из-за несовпадения строки при сравнении с исходными метками такие ответы считаются ошибочными. Это демонстрирует, что T5 может адекватно понимать содержание, но не всегда соблюдает формат вывода.

Модель слишком мала для полноценного few-shot
flan-t5-small — компактная модель, которая с трудом справляется с большим числом примеров в подсказке. Она не всегда запоминает несколько образцов и инструкцию одновременно. Особенно если few-shot примеры получены случайно и не охватывают равномерно все классы. В этом случае модель не получает достаточно разнообразных примеров и начинает делать неверные обобщения.

Есть пути улучшения точности
Несколько простых шагов могут заметно улучшить результат:

Ограничить число примеров до 1–2 (т.е. использовать one- или two-shot).

Добавить инструкцию с явным форматом вывода (например, “выбери строго одно из: World, Sports, Business, Sci/Tech”).

Использовать модель большего размера (например, flan-t5-base или flan-t5-large) — у неё больше контекст и выше устойчивость к шуму.

Отбирать примеры вручную, чтобы они представляли все 4 категории.

Few-shot промт слишком длинный для T5-small
Именно длина подсказки — основное узкое место. В few-shot режиме она быстро достигает предела длины и часть текста отсекается. В итоге модель теряет либо часть инструкции, либо некоторые примеры. Это снижает точность. One-shot — компромиссный вариант, но даже он немного хуже zero-shot в условиях ограниченного объёма модели.

Рекомендации по применению
Для моделей малого размера вроде flan-t5-small наилучший выбор — это zero- или one-shot с чёткой инструкцией. Few-shot режим оправдан только при использовании моделей с более высокой ёмкостью и большим контекстным окном.